<a href="https://colab.research.google.com/github/panghanwu/scikit-learning_tutorial_Elwing/blob/main/naive_bayes_multinomial_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新聞分類

In [ ]:
import glob
# glob:探測檔案路徑 
glob.glob('drive/My Drive/Colab Notebooks/Elwings_tutorial/*')

['drive/My Drive/Colab Notebooks/Elwings_tutorial/naive_bayes_multinomial.ipynb',
 'drive/My Drive/Colab Notebooks/Elwings_tutorial/regression.ipynb',
 'drive/My Drive/Colab Notebooks/Elwings_tutorial/classification.ipynb',
 'drive/My Drive/Colab Notebooks/Elwings_tutorial/cluster.ipynb',
 'drive/My Drive/Colab Notebooks/Elwings_tutorial/tools.ipynb',
 'drive/My Drive/Colab Notebooks/Elwings_tutorial/chinese_news_trans.zip',
 'drive/My Drive/Colab Notebooks/Elwings_tutorial/chinese_news_test.zip']

In [ ]:
import zipfile
# 解壓縮
path = 'drive/My Drive/Colab Notebooks/Elwings_tutorial/chinese_news_trans.zip'
f = zipfile.ZipFile(path)
f.extractall()
path = 'drive/My Drive/Colab Notebooks/Elwings_tutorial/chinese_news_test.zip'
f = zipfile.ZipFile(path)
f.extractall()

In [ ]:
glob.glob('chinese_news_test/*')

['chinese_news_test/環境',
 'chinese_news_test/體育',
 'chinese_news_test/交通',
 'chinese_news_test/醫藥',
 'chinese_news_test/軍事',
 'chinese_news_test/藝術',
 'chinese_news_test/計算機',
 'chinese_news_test/教育',
 'chinese_news_test/政治',
 'chinese_news_test/經濟']

In [ ]:
import os
import pandas as pd

def read_data(base):
  contents, targets = [], []
  basedn = os.path.join(base, '*')
  for dn in glob.glob(basedn):
    upper = os.path.join(dn, '*.TXT')
    lower = os.path.join(dn, '*.txt')
    for fn in glob.glob(upper) + glob.glob(lower):
      with open(fn, 'r', encoding='utf-8') as f:
        contents.append(f.read())
      target = os.path.split(dn)[-1]
      targets.append(target)

  df = pd.DataFrame({
      'content': contents,
      'target': targets
  })
  return df

train_df = read_data('chinese_news_trans')
test_df = read_data('chinese_news_test')
train_df

,content,target
0,《瞭望》週刊文章：一個全球性的環境問題——垃圾\n新華社北京５月１２日...,環境
1,英國確定溫室效應政策\n新華社倫敦５月２５日電為了對付由於溫室效應引起...,環境
2,【 日 期 】19960610\n【 版 號 】10\n【 標 題 】海南人的藍色樂章...,環境
3,經國務院批准我國長江中上游防護林體系建設工程全\n面展開\n (...,環境
4,【 日 期 】19960105\n【 版 號 】7\n【 標 題 】科學家選出２０種最...,環境
...,...,...
2632,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
2633,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
2634,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
2635,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟


In [ ]:
category = train_df['target'].unique()
category

array(['環境', '體育', '交通', '醫藥', '軍事', '藝術', '計算機', '教育', '政治', '經濟'],
      dtype=object)

In [ ]:
trans = {w:i for i, w in enumerate(category)}
trans

{'交通': 2,
 '政治': 8,
 '教育': 7,
 '環境': 0,
 '經濟': 9,
 '藝術': 5,
 '計算機': 6,
 '軍事': 4,
 '醫藥': 3,
 '體育': 1}

In [ ]:
y_train = train_df['target'].replace(trans)
y_test = test_df['target'].replace(trans)
y_test

0      0
1      0
2      0
3      0
4      0
      ..
96     9
97     9
98     9
99     9
100    9
Name: target, Length: 101, dtype: int64

In [ ]:
# test single
p = train_df['content'][0]
p

'           《瞭望》週刊文章：一個全球性的環境問題——垃圾\n新華社北京５月１２日電５月１４日出版的第２０期\n             《瞭望》週刊將發表署名文章《一個全球性的環境問題》\n             。文章摘要如下：\n                 目前，世界各國拋棄的垃圾越來越多，也越來越難處\n             理。據估計，全球每年新增的垃圾有１００億噸左右，人\n             均２噸。\n                 世界上產生垃圾最多的是美國、聯邦德國、英國和日\n             本。僅美國，每年產生工業垃圾２０億噸以上，城市居民\n             垃圾２·２億噸，丟棄的舊汽車１０００多萬輛，廢輪胎\n             上億隻，全國垃圾佔地面積１·１８萬平方公里。華盛頓\n             曾有一家環保機構驚呼：到９０年代末，美國將有一半以\n             上的城市無處傾倒垃圾！\n                 有人形象地說，垃圾“無孔不入”。就連被認為全球\n             淨土的珠穆朗瑪峰、南極，都有垃圾“光顧”。近年來，\n             垃圾又在太空“安家落戶”。自１９５７年蘇聯發射第一\n             顆人造衛星以來，人類在地球周圍的空間留下了大量的太\n             空廢物。據觀測，在太空軌道上有７２００個１０釐米大\n             小以上的廢物碎塊在運轉，這些高速運轉的太空垃圾，成\n             為危險的“飄遊炸彈”。\n                 垃圾不僅有礙觀瞻，佔用大量土地，更重要的是汙染\n             環境，危害人們健康。\n                 地球上的危險廢物正在以每年５億噸的速度增加。危\n             險廢物主要產於西方工業化國家，其產量佔全世界總產量\n             的百分之九十。堆積如山的危險廢料，日益強烈的環保呼\n             聲，十分高昂的處理代價以及有限的處理能力，使歐美國\n             家紛紛向外謀求出路。目前，每５分鐘就有一船危險廢物\n         

In [ ]:
import jieba
from urllib.request import urlretrieve

if not os.path.exists('dict.txt.big'):
    url = 'https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big'
    urlretrieve(url, 'dict.txt.big')
jieba.set_dictionary('dict.txt.big')  # 繁體大詞典

' '.join(jieba.cut(p))

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.520 seconds.
Prefix dict has been built successfully.


'                      《 瞭望 》 週刊 文章 ： 一個 全球性 的 環境 問題 — — 垃圾 \n 新華社 北京 ５ 月 １ ２ 日電 ５ 月 １ ４ 日 出版 的 第 ２ ０ 期 \n                           《 瞭望 》 週刊 將 發表 署名文章 《 一個 全球性 的 環境 問題 》 \n                           。 文章 摘要 如下 ： \n                                   目前 ， 世界 各國 拋棄 的 垃圾 越來越 多 ， 也 越來越 難處 \n                           理 。 據估計 ， 全球 每年 新增 的 垃圾 有 １ ０ ０ 億噸 左右 ， 人 \n                           均 ２ 噸 。 \n                                   世界 上 產生 垃圾 最多 的 是 美國 、 聯邦德國 、 英國 和 日 \n                           本 。 僅 美國 ， 每年 產生 工業 垃圾 ２ ０ 億噸 以上 ， 城市居民 \n                           垃圾 ２ · ２ 億噸 ， 丟棄 的 舊 汽車 １ ０ ０ ０ 多萬輛 ， 廢輪胎 \n                           上 億隻 ， 全國 垃圾 佔地面積 １ · １ ８ 萬平方公里 。 華盛頓 \n                           曾 有 一家 環保 機構 驚呼 ： 到 ９ ０ 年代 末 ， 美國 將有 一半 以 \n                           上 的 城市 無處 傾倒垃圾 ！ \n                                   有人 形象 地說 ， 垃圾 “ 無孔不入 ” 。 就連 被 認為 全球 \n                           淨土 的 珠穆朗瑪 峰 、 南極 ， 都 有 垃圾 “ 光顧 ” 。 近年來 ， \n                           垃圾 又 在 太空 “ 安家落戶 ” 。 自 １ ９ ５ ７ 年 蘇聯

In [ ]:
def multi_cut(p):
  return ' '.join(jieba.cut(p))

x_train = train_df['content'].apply(multi_cut)
x_test = test_df['content'].apply(multi_cut)
x_test

0      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
1      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
2      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
3      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
4      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
                             ...                        
96     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
97     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
98     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
99     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
100    日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
Name: content, Length: 101, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
vec.vocabulary_['\n']

KeyError: ignored

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.5)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

1.0

In [ ]:
from sklearn.metrics import confusion_matrix
# 混淆矩陣
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
      index=[name + '(原本)' for name in category],
      columns=[name + '(預測)' for name in category])

,環境(預測),體育(預測),交通(預測),醫藥(預測),軍事(預測),藝術(預測),計算機(預測),教育(預測),政治(預測),經濟(預測)
環境(原本),10,0,0,0,0,0,0,0,0,0
體育(原本),0,10,0,0,0,0,0,0,0,0
交通(原本),0,0,10,0,0,0,0,0,0,0
醫藥(原本),0,0,0,10,0,0,0,0,0,0
軍事(原本),0,0,0,0,10,0,0,0,0,0
藝術(原本),0,0,0,0,0,10,0,0,0,0
計算機(原本),0,0,0,0,0,0,10,0,0,0
教育(原本),0,0,0,0,0,0,0,10,0,0
政治(原本),0,0,0,0,0,0,0,0,11,0
經濟(原本),0,0,0,0,0,0,0,0,0,10


In [ ]:
p = input('輸入新聞:')
x_predict = vec.transform([multi_cut(p)])
proba = clf.predict_proba(x_predict)[0]
for w, p in zip(category, proba):
  print(w, ':', round(p, 4))

輸入新聞:週二 (6 日) 美國兩黨持續談判，加上聯準會主席持續喊話國會加大紓困力度，華爾街引頸期待新一輪刺激法案出爐，美股早盤震盪走高。  尾盤時段，美國總統川普喊停新紓困案談判，恐慌指數 VIX 應聲衝高，美股聞聲豬羊變色，四大指數全數收跌，波音重挫，道瓊收跌逾 300 點。  道瓊 15 分鐘走勢圖。(圖片：investing) 道瓊 15 分鐘走勢圖。(圖片：investing) 川普推文表示，他已指示白宮代表們停止 (與民主黨代表間的) 談判，直到美國大選後。本輪談判破局的原因在於，兩黨對紓困金的規模與用途仍沒有共識。  眾議院議長佩洛西 (Nancy Pelosi) 隨後發表聲明回應，川普充分展現他的本色，在國會共和黨議員的全力配合下，以國家作為代價，把自己放在第一位。  Fed 主席鮑爾週二出席全美企業經濟協會 (National Association for Business Economics) 並發表演說。  鮑爾稱，經濟復甦未完成，美國仍可能陷入衰退，呼籲國會採取積極的財政和貨幣刺激措施以促進經濟復甦，研判過度的財政紓困風險似乎很小。  全球新冠肺炎 (COVID-19) 疫情持續蔓延，截稿前，據美國約翰霍普金斯大學 (Johns Hopkins University) 即時統計，全球確診數已飆破 3560 萬例，死亡數突破 104 萬例。美國累計確診超過 747 萬例，累計死亡數超過 21 萬。
環境 : 0.0
體育 : 0.0
交通 : 0.0
醫藥 : 0.0
軍事 : 0.0
藝術 : 0.0
計算機 : 0.0
教育 : 0.0
政治 : 0.0
經濟 : 1.0
